In [1]:
import duckdb

In [2]:
duckdb.sql("INSTALL httpfs; LOAD httpfs;")
duckdb.sql("INSTALL iceberg; LOAD iceberg;")

In [3]:
duckdb.sql("SET s3_endpoint='127.0.0.1:9000';")
duckdb.sql("SET s3_access_key_id='minio';")
duckdb.sql("SET s3_secret_access_key='minio123';")
duckdb.sql("SET s3_region='us-east-1';")
duckdb.sql("SET s3_url_style='path';")
duckdb.sql("SET s3_use_ssl=false;")

In [4]:
result = duckdb.sql("""
SELECT * FROM iceberg_scan('s3://warehouse/goodwiki/ingestion_etl')
""").df()

In [5]:
result

,pageid,title,revid,description,categories,markdown
0,57185536,Georgia Hopley,1163683705,American journalist and temperance advocate,"[1858 births, 1944 deaths, 19th-century Americ...",Georgianna Eliza Hopley (1858–1944) was an Ame...
1,15394015,Willis Ward,1170257280,Track and field athlete and American football ...,"[1912 births, 1983 deaths, 20th-century Africa...","Willis Franklin Ward (December 28, 1912 – Dece..."
2,62958021,Instant Replay Game,1166130448,Notable American football game,"[1989 National Football League season, 1989 in...","The Instant Replay Game, also known as the Ast..."
3,17546,Louvre,1172724876,"Art museum in Paris, France","[1793 establishments in France, Archaeological...","The Louvre (English: /ˈluːv(rə)/ ), or the Lou..."
4,13280513,Launch Party,1150875056,None,"[2007 American television episodes, The Office...","""Launch Party"" is the fifth and sixth episode ..."
...,...,...,...,...,...,...
44749,65986818,Let Us Continue,1148889149,1963 speech by U.S. President Lyndon B. Johnson,"[1963 in American politics, 1963 in Washington...",Let Us Continue was a speech that 36th Preside...
44750,23240578,Odin-class coastal defense ship,1161437920,Coastal defense ship class of the German Imper...,"[Coastal defense ship classes, Odin-class coas...",The Odin class was a pair of coastal defense s...
44751,209236,Pest (organism),1173463954,Organism harmful to humans/our concerns,"[Insects in culture, Pests (organism)]",A pest is any organism harmful to humans or hu...
44752,62440986,44 Union Square,1170828953,"Office building in Manhattan, New York","[1929 establishments in New York City, New Yor...","44 Union Square, also known as 100 East 17th S..."


In [6]:
#creating sqlalchemy connection for m-schema
from sqlalchemy import create_engine
db_engine = create_engine(f"duckdb:///:memory")

In [ ]:
#move the dataframe into duckdb warehouse for mschema
result.to_sql('goodwiki', db_engine, if_exists='replace', index=False)

/Users/tommac/Documents/personal_project/t2sql-lakehouse/.venv/lib/python3.11/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


-1

In [ ]:
#have schema engine conenct to duckdb memory to generate mschema
from schema_engine import SchemaEngine

db_name = ":memory"

schema_engine = SchemaEngine(engine=db_engine, db_name=db_name)
mschema = schema_engine.mschema
mschema_str = mschema.to_mschema()
print(mschema_str)
mschema.save(f'./{db_name}.json')

/Users/tommac/Documents/personal_project/t2sql-lakehouse/.venv/lib/python3.11/site-packages/duckdb_engine/__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


【DB_ID】 :memory
【Schema】
# Table: ":memory".main.goodwiki
[
(pageid:BIGINT, Examples: [7750611, 32300234, 656951]),
(title:VARCHAR, Examples: [Instant Replay Game]),
(revid:BIGINT, Examples: [1171051001, 1172990158, 1149830414]),
(description:VARCHAR, Examples: [North Indian cyclone in 2001]),
(categories:VARCHAR),
(markdown:VARCHAR)
]


In [57]:
dialect = db_engine.dialect.name
question = ''
evidence = ''
prompt = """You are now a {dialect} data analyst, and you are given a database schema as follows:

【Schema】
{db_schema}

【Question】
{question}

【Evidence】
{evidence}

[Question]
Can you get me the description of the louvre?

Please read and understand the database schema carefully, and generate an executable SQL based on the user's question and evidence. The generated SQL is protected by ```sql and ```.
""".format(dialect=dialect, question=question, db_schema=mschema_str, evidence=evidence)


In [58]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "XGenerationLab/XiYanSQL-QwenCoder-7B-2504"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

## dialects -> ['SQLite', 'PostgreSQL', 'MySQL']
message = [{'role': 'user', 'content': prompt}]

text = tokenizer.apply_chat_template(
    message,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    do_sample=True,
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.17s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [61]:
response

"SELECT description FROM goodwiki WHERE title = 'Louvre';"

In [62]:
from sqlalchemy import text
with db_engine.connect() as conn:
    result = conn.execute(text(response))
    for row in result:
        print(row)

('Art museum in Paris, France',)
